# HW 6: Sentiment Analysis & Word Vectors

In this question, you'll need dataset:
- `hw6_train.csv`: dataset for training
- `hw6_test.csv`: dataset for test

A snippet of the dataset is given below.



In [1]:
import pandas as pd

train = pd.read_csv("hw6_train.csv")
test = pd.read_csv("hw6_test.csv")

train

,label,text
0,1,Getting ready for college. I had a good sleep....
1,1,We are having a party now to have all the fami...
2,0,@marC0110 ummm.. i see you.. and i really wann...
3,1,@saboteur1 Thanks for following Much apprecia...
4,1,Why eat at home? Picnic plans for today are al...
...,...,...
59995,0,@kaylacelina why so sad?
59996,1,"@annahawkins Plus, we have Panera and Powells."
59997,0,About to ge ready for work as i have a 9 hour ...
59998,0,i think I may have broken my finger!


## Q1: Unsupervised Sentiment Analysis (4 points)

- Write a function `analyze_sentiment(docs, labels, th)` as follows: (3 points)
    - Takes three inputs:
       - `docs` : a list of documents, 
       - `labels` the ground-truth sentiment labels of `docs`
       - `th`: compound threshold
    - Use Vader to get a compound score of for each document in `docs`.  
    - If `compound score > th`, then the predicted label is 1; otherwise 0
    - Print out the classification report
    - Return F1 macro score


- Tune `th` such that the F1 macro score is maximimized (1 point)
- With the `th` tuned, calculate the performance on the test dataset.

In [18]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import word2vec
from gensim.models import Word2Vec

import logging
import nltk
import string
from sklearn.svm import LinearSVC

In [3]:
def analyze_sentiment(docs, labels, th=0):
    sia = SentimentIntensityAnalyzer()
    compound_scores = [sia.polarity_scores(doc)['compound'] for doc in docs]
    predicted_labels = [1 if score > th else 0 for score in compound_scores]
    print(classification_report(labels, predicted_labels))
    f1 = f1_score(labels, predicted_labels, average='macro')
                
    return f1

In [8]:

# Tune threshold

# use your selected threshold to calculate performance 

train_docs = list(train.text)
train_labels = list(train.label)
test_docs = list(test.text)
test_labels = list(test.label)

best_f1 = 0
best_th = 0

for th in np.arange(-1, 1.01, 0.1):
    f1 = analyze_sentiment(train_docs, train_labels, th)
    if f1 > best_f1:
        best_f1 = f1
        best_th = th


print("best th: ", best_th, "best f1: ", best_f1)

c:\Users\kshag\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\kshag\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\kshag\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     29888
           1       0.50      1.00      0.67     30112

    accuracy                           0.50     60000
   macro avg       0.25      0.50      0.33     60000
weighted avg       0.25      0.50      0.34     60000

              precision    recall  f1-score   support

           0       0.91      0.00      0.01     29888
           1       0.50      1.00      0.67     30112

    accuracy                           0.50     60000
   macro avg       0.71      0.50      0.34     60000
weighted avg       0.71      0.50      0.34     60000

              precision    recall  f1-score   support

           0       0.89      0.03      0.05     29888
           1       0.51      1.00      0.67     30112

    accuracy                           0.51     60000
   macro avg       0.70      0.51      0.36     60000
weighted avg       0.70      0.51      0.36     60000

              preci

c:\Users\kshag\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\kshag\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\kshag\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

              precision    recall  f1-score   support

           0       0.50      1.00      0.67     29888
           1       0.00      0.00      0.00     30112

    accuracy                           0.50     60000
   macro avg       0.25      0.50      0.33     60000
weighted avg       0.25      0.50      0.33     60000

best th:  0.19999999999999973 best f1:  0.6624565997472943


In [10]:

for th in np.arange(best_th-.1, best_th+.1, 0.01):
    f1=analyze_sentiment(train_docs, train_labels, th)
    if f1 > best_f1:
        best_f1 = f1
        best_th = th
        
print("best th: ", best_th, "best f1: ", best_f1)


              precision    recall  f1-score   support

           0       0.64      0.71      0.67     29888
           1       0.68      0.61      0.64     30112

    accuracy                           0.66     60000
   macro avg       0.66      0.66      0.66     60000
weighted avg       0.66      0.66      0.66     60000

              precision    recall  f1-score   support

           0       0.64      0.71      0.67     29888
           1       0.68      0.61      0.64     30112

    accuracy                           0.66     60000
   macro avg       0.66      0.66      0.66     60000
weighted avg       0.66      0.66      0.66     60000

              precision    recall  f1-score   support

           0       0.64      0.71      0.67     29888
           1       0.68      0.61      0.64     30112

    accuracy                           0.66     60000
   macro avg       0.66      0.66      0.66     60000
weighted avg       0.66      0.66      0.66     60000

              preci

In [11]:

test_f1 = analyze_sentiment(test_docs, test_labels, best_th)
print("test f1: ", test_f1)

              precision    recall  f1-score   support

           0       0.64      0.74      0.69      9968
           1       0.69      0.58      0.63     10032

    accuracy                           0.66     20000
   macro avg       0.66      0.66      0.66     20000
weighted avg       0.66      0.66      0.66     20000

test f1:  0.6578496050005054


## Q2: Supervised Sentiment Analysis Using Word Vectors (6 points)

### Q2.1: Train Word Vectors

Write a function `train_wordvec(docs, vector_size)` as follows: (2 points)
- Take two inputs:
    - `docs`: a list of documents
    - `vector_size`: the dimension of word vectors
- First tokenize `docs` into tokens, remove punctuations and only keep tokens with at least two characters
- Use `gensim` package to train word vectors. Set the `vector size` and also carefully set other parameters such as `window`, `min_count` etc. Explain how you select these parameters. 
- return the trained word vector model

In [15]:
def tokenize(doc):
    tokens = nltk.word_tokenize(doc)
    okens = [token.lower() for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if len(token) > 1]
    return tokens

In [19]:
def train_wordvec(docs, vector_size = 100):
    tokenized_docs = [tokenize(doc) for doc in docs]
    wv_model = Word2Vec(
        sentences=tokenized_docs,
        vector_size=vector_size,
        window=5,  
        min_count=2,
        workers=4, 
        sg=1  
    )   
    
    return wv_model

In [20]:
wv_model = train_wordvec(train["text"], vector_size = 100)

### Q2.2: Generate Vector Representation for Documents (2.5 points)

Write a function `generate_doc_vector(train_docs, test_docs, wv_model, wv_dim= 100, stop_words = None, min_df = 1)` as follows:
- Take two inputs:
    - `train_docs`: a list of train documents, 
    - `test_docs`: a list of train documents, 
    - `wv_model`: trained word vector model. 
    - `wv_dim`: dimensionality of word vector. Set the default value to 100.
    - `stop_words`: whether to remove stopwords
    - `min_df`: minimum document frequency
- First vectorize each document using TFIDF vectorizer by considering stop_words and min_df configurations.
- For each token in the vocabulary, look up for its word vector in `wv_model`. 
- Then the document vector (denoted as `d`) of `doc` can be calculated as the `TFIDF-weighted sum of the word vectors of its tokens`, i.e. $d = \frac{1}{\sum{tfidf_i}} * \sum_{i \in doc}{tfidf_i * v_i}$, where $v_i$ is the word vector of the i-th token, and $tfidf_i$ is the tfidf weigth of this token.
- Return the vector representations of all `train_docs` as a numpy array of shape `(n, vector_size)`, where `n` is the number of documents in `train_docs` and `vector_size` is the dimension of word vectors. Create similar representations for `test_docs` (HINT: here you need to `transform` test documents so that they are on the same tfidf dimensions as train documents and then generate the vector representations).



In [21]:
def generate_doc_vector(train_docs, test_docs, wv_model, wv_dim= 100,
                        stop_words = None, min_df = 1):
    
    vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=min_df)

    tfidf_train = vectorizer.fit_transform(train_docs)
    tfidf_test = vectorizer.transform(test_docs)
    vocab = vectorizer.get_feature_names_out()

    train_vectors = []
    test_vectors = []

    def compute_vectors(tfidf_matrix, docs):
        vectors = []
        for i, doc in enumerate(docs):
            tfidf_row = tfidf_matrix[i]
            feature_indices = tfidf_row.indices
            tfidf_values = tfidf_row.data
            weighted_sum = np.zeros(wv_dim)
            weight_sum = 0
            for j, idx in enumerate(feature_indices):
                token = vocab[idx]
                if token in wv_model.wv:
                    weighted_sum += tfidf_values[j] * wv_model.wv[token]
                    weight_sum += tfidf_values[j]

            if weight_sum > 0:
                vectors.append(weighted_sum / weight_sum)
            else:
                vectors.append(np.zeros(wv_dim))
        return vectors

    train_vec = compute_vectors(tfidf_train, train_docs)
    test_vec = compute_vectors(tfidf_test, test_docs)
    train_vec=np.array(train_vec)
    test_vec = np.array(test_vec)
   
    return train_vec, test_vec

In [22]:
train_X, test_X = generate_doc_vector(train["text"], test["text"], 
                                      wv_model, wv_dim= 100,
                                      stop_words = None, min_df = 5)


### Q2.3: Put everything together (1 point)


Define a function `predict_sentiment(train_text, train_label, test_text, test_label, wv_model, wv_dim= 100, stop_words = None, min_df = 1)` as follows:

- Take the following inputs:
    - `train_text, train_label`: a list of documents and their labels for training
    - `test_text, test_label`: a list of documents and their labels for testing,
    - `wv_model`: trained word vector model. 
    - `wv_dim`: dimensionality of word vector. Set the default value to 100.
    - `stop_words`: whether to remove stopwords
    - `min_df`: minimum document frequency
- Call `generate_doc_vector` to generate vector representations (denoted as `train_X` and `test_X`) for documents in `train_text` and `test_text`. 
- Fit a linear SVM model using `train_X` and `train_label`
- Predict the label for `test_X` and print out classification report for the testing subset.
- This function has no return

In [23]:
def predict_sentiment(train_text, train_label, 
                      test_text, test_label, 
                      wv_model, wv_dim= 100,
                      stop_words = None, min_df = 1):
    train_X, test_X = generate_doc_vector(train_text, test_text, wv_model, wv_dim=wv_dim, stop_words=stop_words, min_df=min_df)

    svm_model = LinearSVC()
    svm_model.fit(train_X, train_label)

    predicted_labels = svm_model.predict(test_X)

    print(classification_report(test_label, predicted_labels))   

### Q2.4: Analysis (0.5 point)

- Compare the classification reports you obtain from Q1 and Q2.3. Which model performs better?
- Why this model can achieve better performance? Please explain. 

In [24]:
predict_sentiment(train["text"], train["label"],\
                  test["text"], test["label"],\
                  wv_model, wv_dim= 100,
                  stop_words = None, min_df = 5)
    

c:\Users\kshag\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.73      0.71      0.72      9968
           1       0.72      0.73      0.73     10032

    accuracy                           0.72     20000
   macro avg       0.72      0.72      0.72     20000
weighted avg       0.72      0.72      0.72     20000



The Word2Vec-based model outperforms the Vader-based sentiment analysis from Q1, as we can see by the higher accuracy (.72 in Q2.3 compared to .66 in Q1) and f1 score (.72 in Q2.3 compared to .6578 in Q1) . The Word2Vec model achieves better performance due to its ability to better capture contextual word relationships, which provides a more detailed representation of the text data. Additionally, tf-idf weighting highlights important words while reducing the influence of common but not important words. The SVM classifier is also specifically optimized for sentiment classification, compared to the unsupervised Vader tool, which also helps to give us improved results in Q2.3.